In [1]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import torch
torch.cuda.empty_cache()

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import transformers
import pytorch_lightning as pl


from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from typing import Sequence
from functools import partial
from collections import Counter

print(torch.__version__)
print(transformers.__version__)
print(torch.cuda.is_available())

1.10.1+cu111
4.27.3
True


In [2]:
# Читаем данные из датасетов
movies_df = pd.read_csv('../input/movies.csv')
ratings_df  = pd.read_csv('../input/ratings.csv')
tags_df  = pd.read_csv('../input/tags.csv')


In [3]:
train_data, test_data = train_test_split(ratings_df, test_size=0.2, random_state=42) # сплитим данные

In [4]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [5]:
from typing import Any, Optional


from pytorch_lightning.utilities.types import STEP_OUTPUT


class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512)

In [6]:
num_users = ratings_df['userId'].max()+1
num_items = ratings_df['movieId'].max()+1

all_movieIds = ratings_df['movieId'].unique()

model = NCF(num_users, num_items, train_data, all_movieIds)

In [13]:
checkpoint = torch.load('models/with_val.pth')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [14]:
ratings_df_copy = ratings_df.copy()
test_data_copy = test_data.copy()

test_data_copy.sample(frac=1)
test_data_copy.drop(test_data_copy.index[int(len(test_data_copy)/1000):len(test_data_copy)-1], axis=0, inplace=True)

# Создаем множество пользователей и фильмов из тестовых данных
test_user_item_set = set(zip(test_data_copy['userId'], test_data_copy['movieId']))

# Группируем идентификаторы фильмов для каждого пользователя в словаре
user_interacted_items = ratings_df_copy.groupby('userId')['movieId'].apply(list).to_dict()

# Создаем список для хранения результатов оценки качества предсказаний
hits = []

# Для каждой пары (пользователь, фильм) в тестовых данных
for (u,i) in tqdm(test_user_item_set):

    # Получаем список идентификаторов фильмов, с которыми взаимодействовал данный пользователь
    interacted_items = user_interacted_items[u]

    # Создаем множество идентификаторов фильмов, с которыми пользователь не взаимодействовал
    not_interacted_items = set(all_movieIds) - set(interacted_items)

    # Случайным образом выбираем 99 не взаимодействовавших фильмов и добавляем целевой фильм
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    # Получаем предсказанные оценки для каждого из выбранных фильмов
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    # Получаем идентификаторы 10 фильмов с наивысшими предсказанными оценками
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    # Проверяем, содержится ли целевой фильм в топ-10 предсказанных
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
# Выводим значение метрики Hit Ratio конкретно здесь для топ 10 у юзера (метрика для рекомендательной систем не знаю даже почему нужно обосновывать)        
print("The Hit Ratio @ 10 is {:.10f}".format(np.average(hits)))

100%|██████████| 5551/5551 [01:49<00:00, 50.54it/s]

The Hit Ratio @ 10 is 0.76
